<h2><u>EXERCISE 1</u></h2>
<h3><u>TASK1 1</u></h3>

In [1]:
import numpy as np
import json
import ijson # library which will be used to read the review dataset line by line, without loading it in memory


business_dict = {} # dictionary with business id's as keys and the list [category, review_texts] as values

with open('yelp_academic_dataset_business.json', 'r',encoding = "utf8") as f:
    for line in f.readlines():
        dictionary = json.loads(line)
        if((dictionary['city'] == 'Toronto') and (dictionary['review_count'] >= 10)):
            categories = dictionary['categories']
            categ_lst = []

            if(categories):
                categories = categories.split(', ')
                
                for i in range(len(categories)):
                    if(categories[i] == "Beauty & Spas" or categories[i] == "Shopping" or categories[i] == "Bars"):
                
                        categ_lst.append(categories[i])

            if(not categ_lst): # if list is empty, then none of the wanted categories have been found
                continue
            else: # priority order: "Beauty & Spas" first, "Shopping" second, "Bars" third in case more than one is present in the attributes
                if("Beauty & Spas" in categ_lst):
                    business_dict[dictionary["business_id"]] = ["Beauty & Spas", ""]
                elif("Shopping" in categ_lst):
                    business_dict[dictionary["business_id"]] = ["Shopping", ""]
                else:
                    business_dict[dictionary["business_id"]] = ["Bars", ""]

with open('yelp_academic_dataset_review.json ', 'r', encoding = "utf8") as f1:   
    parser = ijson.parse(f1, multiple_values = True) 

    business_id = ''  
    
    for prefix, event, value in parser:
        if(prefix == "business_id"):
            business_id = value
        if(prefix == "text"):
            if(business_id in business_dict.keys()):
                new_text = value
                previous_text = business_dict[business_id][1]
                joined_text = ' '.join([previous_text, new_text])
                business_dict[business_id][1] = joined_text
            business_id = ''

After storing the businesses that match the requirements, I then extract the features of their reviews using the tf-idf representation. I remove stop words, non-english words, very rare words and very common words. 

In [2]:
import sklearn as sk
import sklearn.feature_extraction.text as sk_text

corpus = []

for key in business_dict.keys():
    corpus.append(business_dict[key][1])

vectorizer = sk_text.TfidfVectorizer(token_pattern=r'[a-z]+', stop_words = 'english', min_df= 10, max_df = 0.7)
test = vectorizer.fit_transform(corpus)
print(len(vectorizer.get_feature_names()))
print(vectorizer.get_feature_names())

18963
['aa', 'aaa', 'aaron', 'ab', 'aback', 'abandon', 'abandoned', 'abbey', 'abd', 'aber', 'abhorrent', 'abide', 'abilities', 'ability', 'abit', 'able', 'abnormal', 'abnormally', 'abomination', 'abound', 'abrasive', 'abroad', 'abrupt', 'abruptly', 'abs', 'absence', 'absent', 'absinthe', 'absolute', 'absolutely', 'absorb', 'absorbed', 'absorbing', 'abstract', 'absurd', 'absurdity', 'absurdly', 'abt', 'abundance', 'abundant', 'aburi', 'abuse', 'abused', 'abusing', 'abusive', 'abut', 'abv', 'abysmal', 'ac', 'academic', 'academy', 'acc', 'accent', 'accented', 'accents', 'accentuate', 'accentuated', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessed', 'accessibility', 'accessible', 'accessories', 'accessory', 'accident', 'accidental', 'accidentally', 'accidently', 'accidents', 'acclaimed', 'accolades', 'accommodate', 'accommodated', 'accommodates', 'accommodating', 'accommodation', 'accommodations', 'accomodate', 'accomodated', 'accomodating', 'ac

In [3]:
from sklearn.model_selection import train_test_split

business_categories = []
business_dict_values = list(business_dict.values())

for i in range(len(business_dict_values)): # indexing of each business clustering category
    if(business_dict_values[i][0] == "Beauty & Spas"):
        business_categories.append(0)
    elif(business_dict_values[i][0] == "Shopping"):
        business_categories.append(1)
    else:
        business_categories.append(2)
    
print(business_categories)

[2, 2, 0, 1, 1, 1, 1, 2, 2, 1, 0, 0, 1, 1, 2, 0, 2, 0, 2, 2, 1, 2, 0, 2, 0, 2, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 0, 0, 1, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 1, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 0, 0, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 0, 1, 2, 0, 1, 2, 1, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 1, 2, 2, 0, 0, 2, 2, 0, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 0, 0, 1, 0, 2, 2, 1, 0, 2, 1, 1, 2, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 1, 1, 2, 0, 0, 1, 0, 2, 2, 1, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 0, 0, 2, 2, 2, 1, 1, 2, 2, 0, 1, 0, 1, 2, 1, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 2, 1, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 0, 2, 0, 2, 0, 0, 1, 2, 1, 1, 1, 2, 1, 2, 0, 2, 2, 0, 2, 0, 1, 2, 2, 1, 0, 2, 2, 0, 1, 2, 0, 2, 1, 1, 1, 2, 2, 0, 1, 2, 1, 2, 1, 1, 2, 0, 0, 1, 0, 1, 0, 1, 1, 1, 2, 2, 1, 2, 2, 1, 0, 1, 0, 0, 0, 2, 

I then create the business vectors, using the tf-idf vectoriser.

In [4]:
X = vectorizer.transform(corpus)
print(X.shape)

(2991, 18963)


Splitting the dataset into training and testing set.

In [5]:
y = business_categories
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
print(X_train.shape[0], X_test.shape[0])
print(len(y_train), len(y_test))
print(y_train)

1794 1197
1794 1197
[0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 2, 1, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 2, 2, 1, 1, 1, 1, 0, 2, 2, 1, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 0, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 2, 0, 2, 0, 1, 1, 0, 0, 1, 2, 2, 0, 2, 2, 1, 0, 2, 1, 2, 1, 0, 2, 0, 0, 2, 0, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 1, 1, 2, 0, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 0, 2, 1, 0, 2, 2, 0, 0, 2, 0, 2, 2, 1, 0, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 0, 2, 0, 0, 1, 2, 2, 0, 1, 2, 0, 0, 1, 2, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 1, 0, 1, 1, 0, 2, 0, 2, 1, 1, 0, 2, 0, 2, 2, 0, 1, 2, 1, 1, 1, 2, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 2, 1, 0, 0, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 2, 2, 2, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 2, 0, 2, 2, 1, 2, 0, 1, 2, 0, 1, 0, 0, 1, 2, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2

I then declare a function which takes a training model as an argument, along with the training and testing set and then uses the model on the data, producing and printing all the metrics that were asked. This is done in order to avoid rewriting the same code.

In [6]:
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics

lr_clf = linear_model.LogisticRegression(solver='lbfgs')

def ApplyModelAndPrintScores(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    print("classifier score:",model.score(X_test,y_test))
    y_pred = model.predict(X_test)
    print("\naccuracy:",metrics.accuracy_score(y_test,y_pred))
    print("\nConfusion matrix")
    print(metrics.confusion_matrix(y_test,y_pred))
    print("\nPrecision Score per class")
    print(metrics.precision_score(y_test,y_pred,average=None))
    print("\nAverage Precision Score")
    print(metrics.precision_score(y_test,y_pred,average='weighted'))
    print("\nRecall Score per class")
    print(metrics.recall_score(y_test,y_pred,average=None))
    print("\nAverage Recall Score")
    print(metrics.recall_score(y_test,y_pred,average='weighted'))
    print("\nF1-score Score per class")
    print(metrics.f1_score(y_test,y_pred,average=None))
    print("\nAverage F1 Score")
    print(metrics.f1_score(y_test,y_pred,average='weighted'))
    
ApplyModelAndPrintScores(lr_clf, X_train, y_train, X_test, y_test)

classifier score: 0.9766081871345029

accuracy: 0.9766081871345029

Confusion matrix
[[307  14   0]
 [  0 356  13]
 [  1   0 506]]

Precision Score per class
[0.99675325 0.96216216 0.97495183]

Average Precision Score
0.976855645848214

Recall Score per class
[0.95638629 0.96476965 0.99802761]

Average Recall Score
0.9766081871345029

F1-score Score per class
[0.97615262 0.96346414 0.98635478]

Average F1 Score
0.9765623486428892


In [7]:
from sklearn import svm

svm_clf = svm.SVC()
ApplyModelAndPrintScores(svm_clf, X_train, y_train, X_test, y_test)

classifier score: 0.9757727652464494

accuracy: 0.9757727652464494

Confusion matrix
[[308  13   0]
 [  1 354  14]
 [  1   0 506]]

Precision Score per class
[0.99354839 0.96457766 0.97307692]

Average Precision Score
0.975946689700431

Recall Score per class
[0.95950156 0.95934959 0.99802761]

Average Recall Score
0.9757727652464494

F1-score Score per class
[0.97622821 0.96195652 0.98539435]

Average F1 Score
0.9757110679875979


In [8]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
ApplyModelAndPrintScores(knn, X_train, y_train, X_test, y_test)

classifier score: 0.9632414369256475

accuracy: 0.9632414369256475

Confusion matrix
[[313   8   0]
 [  9 339  21]
 [  3   3 501]]

Precision Score per class
[0.96307692 0.96857143 0.95977011]

Average Precision Score
0.9633700899970021

Recall Score per class
[0.97507788 0.91869919 0.98816568]

Average Recall Score
0.9632414369256475

F1-score Score per class
[0.96904025 0.94297636 0.97376093]

Average F1 Score
0.9630050024143985


In [9]:
from sklearn.naive_bayes import GaussianNB

naive_bayes_model = GaussianNB()
ApplyModelAndPrintScores(naive_bayes_model, X_train.toarray(), y_train, X_test.toarray(), y_test)

classifier score: 0.8554720133667502

accuracy: 0.8554720133667502

Confusion matrix
[[271  20  30]
 [  2 247 120]
 [  0   1 506]]

Precision Score per class
[0.99267399 0.92164179 0.77134146]

Average Precision Score
0.8770294857937291

Recall Score per class
[0.84423676 0.66937669 0.99802761]

Average Recall Score
0.8554720133667502

F1-score Score per class
[0.91245791 0.7755102  0.87016337]

Average F1 Score
0.8523267202137944


In [10]:
from sklearn import tree

dtree = tree.DecisionTreeClassifier()
ApplyModelAndPrintScores(dtree, X_train, y_train, X_test, y_test)

classifier score: 0.9197994987468672

accuracy: 0.9197994987468672

Confusion matrix
[[303  16   2]
 [ 27 321  21]
 [  1  29 477]]

Precision Score per class
[0.91540785 0.87704918 0.954     ]

Average Precision Score
0.9199290467762193

Recall Score per class
[0.94392523 0.8699187  0.9408284 ]

Average Recall Score
0.9197994987468672

F1-score Score per class
[0.92944785 0.87346939 0.94736842]

Average F1 Score
0.91978174961697


Using cross validation is the best way to determine how good a model is, since it splits the dataset into numerous parts and uses each part as testing set once, which means that practically the model is tested on all the data. This helps us assess its overall performance better.

In [11]:
import sklearn.model_selection as model_selection

def show_five_cross_validation_scores(method_name, method_obj, X, y):
    print("Evaluating method", method_name, "using 5-fold cross validation:")
    scores = model_selection.cross_val_score(method_obj, X, y, scoring='f1_weighted', cv=5)
    print ("F1 weighted scores:", scores)
    print ("F1 weighted mean score:", scores.mean())
    scores = model_selection.cross_validate(method_obj, X, y, scoring=['precision_weighted','recall_weighted'], cv=3)
    print ("Precision weighted scores:", scores['test_precision_weighted'])
    print ("Precision weighted mean score:",scores['test_precision_weighted'].mean())
    print ("Recall weighted scores:", scores['test_recall_weighted'])
    print ("Recall weighted mean score:", scores['test_recall_weighted'].mean())
    print ("\n")
    
show_five_cross_validation_scores("Logistic Regression", lr_clf, X, y)

Evaluating method Logistic Regression using 5-fold cross validation:
F1 weighted scores: [0.96991144 0.94988803 0.96310581 0.97152794 0.97661758]
F1 weighted mean score: 0.9662101622556225
Precision weighted scores: [0.96045417 0.96175147 0.9770586 ]
Precision weighted mean score: 0.9664214153008294
Recall weighted scores: [0.95987964 0.96088265 0.97693079]
Recall weighted mean score: 0.9658976930792377




In [12]:
show_five_cross_validation_scores("k-Nearest Neighbours", knn, X, y)

Evaluating method k-Nearest Neighbours using 5-fold cross validation:
F1 weighted scores: [0.96149001 0.9476806  0.95776684 0.94805959 0.95631664]
F1 weighted mean score: 0.9542627328956664
Precision weighted scores: [0.95818113 0.94508797 0.95369361]
Precision weighted mean score: 0.9523209017651181
Recall weighted scores: [0.95787362 0.9448345  0.95386158]
Recall weighted mean score: 0.9521899030424607




In [13]:
show_five_cross_validation_scores("Naive Bayes", naive_bayes_model, X.toarray(), y)

Evaluating method Naive Bayes using 5-fold cross validation:


c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\naive_bayes.py", line 212, in fit
    return self._partial_fit(X, y, np.unique(y), _refit=True,
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\naive_bayes.py", line 372, in _partial_fit
    self.epsilon_ = self.var_smoothing * np.var(X, axis=0).max()
  File "<__array_function__ internals>", line 5, in var
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\numpy\cor

F1 weighted scores: [nan nan nan nan nan]
F1 weighted mean score: nan


c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\naive_bayes.py", line 212, in fit
    return self._partial_fit(X, y, np.unique(y), _refit=True,
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\naive_bayes.py", line 372, in _partial_fit
    self.epsilon_ = self.var_smoothing * np.var(X, axis=0).max()
  File "<__array_function__ internals>", line 5, in var
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\numpy\cor

Precision weighted scores: [nan nan nan]
Precision weighted mean score: nan
Recall weighted scores: [nan nan nan]
Recall weighted mean score: nan




c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\naive_bayes.py", line 212, in fit
    return self._partial_fit(X, y, np.unique(y), _refit=True,
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\naive_bayes.py", line 372, in _partial_fit
    self.epsilon_ = self.var_smoothing * np.var(X, axis=0).max()
  File "<__array_function__ internals>", line 5, in var
  File "c:\users\j\appdata\local\programs\python\python38-32\lib\site-packages\numpy\cor

In [14]:
show_five_cross_validation_scores("D tree", dtree, X, y)

Evaluating method D tree using 5-fold cross validation:
F1 weighted scores: [0.92845274 0.9032547  0.91583908 0.92288489 0.93766936]
F1 weighted mean score: 0.9216201544909193
Precision weighted scores: [0.91343381 0.91749155 0.92301321]
Precision weighted mean score: 0.9179795251331516
Recall weighted scores: [0.91374122 0.91775326 0.9227683 ]
Recall weighted mean score: 0.9180875961216984




In [15]:
show_five_cross_validation_scores("Support Vector Machines", svm_clf, X, y)

Evaluating method Support Vector Machines using 5-fold cross validation:
F1 weighted scores: [0.96818635 0.953282   0.95978282 0.97149441 0.97502201]
F1 weighted mean score: 0.9655535201489194
Precision weighted scores: [0.96330958 0.96183914 0.97517914]
Precision weighted mean score: 0.966775952326158
Recall weighted scores: [0.96288867 0.96088265 0.97492477]
Recall weighted mean score: 0.9662320294215981




<h3>TASK 2</h3>

In [16]:
import gensim 
from gensim.models import Word2Vec

In [17]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

C:\Users\J/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


I use the parameter limit = 100000 because otherwise I run into memory problems.

In [18]:
path = r"C:\Users\J/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz"
g_model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True, limit = 100000)  

In [19]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\J\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
print(len(corpus))
print(len(business_categories))

2991
2991


I turn all words into lowercase words.

In [21]:
import string as string

X_nltk = []

for review in corpus:
    wt = word_tokenize(review.lower())
    review = [w for w in wt if w not in string.punctuation]
    X_nltk.append(review)


In [22]:
print(X_nltk[0])

['consistently', 'good', 'as', 'the', 'keg', 'tends', 'to', 'be', 'highlights', 'great', 'lunchtime', 'filet', 'and', 'side', 'vegetables', 'great', 'service', 'that', 'checked', 'in', 'with', 'just', 'the', 'right', 'frequency', 'bowls', 'and', 'bowls', 'of', 'those', 'great', 'chocolate', 'mints', 'scattered', 'throughout', 'the', 'restaurant', 'so', 'that', 'you', 'can', 'pocket', 'a', 'few', 'extra', 'directly', 'across', 'from', 'the', 'south', 'entrance/exit', 'of', 'the', 'eglinton', 'ttc', 'stop', 'medium', 'parking', 'available', 'underground', 'but', 'it', 'was', 'a', 'bit', 'expensive', 'side', 'caesar', 'was', "n't", 'served', 'a', 'lemon', 'wedge', 'low', 'opens', 'exactly', 'at', '11:30', 'am', 'or', 'shortly', 'after', 'so', 'if', 'you', 'arrive', 'a', 'bit', 'early', 'be', 'prepared', 'to', 'wait', 'outside', 'and', 'not', 'in', 'the', 'lounge', 'second', 'visit', 'to', 'the', 'keg', 'steakhouse', 'yonge/eglinton', 'after', 'being', 'unimpressed', 'last', 'time', 'this'

I remove all stop words using the nltk library:

In [23]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\J\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
english_stop_words = set(stopwords.words('english'))
X_nltk_new = []

for x in X_nltk:
    review = [w for w in x if (w not in english_stop_words)]
    if len(review) == 0: continue    
    X_nltk_new.append(review)

In [25]:
print(len(X_nltk_new))
print(X_nltk_new[0])

2991
['consistently', 'good', 'keg', 'tends', 'highlights', 'great', 'lunchtime', 'filet', 'side', 'vegetables', 'great', 'service', 'checked', 'right', 'frequency', 'bowls', 'bowls', 'great', 'chocolate', 'mints', 'scattered', 'throughout', 'restaurant', 'pocket', 'extra', 'directly', 'across', 'south', 'entrance/exit', 'eglinton', 'ttc', 'stop', 'medium', 'parking', 'available', 'underground', 'bit', 'expensive', 'side', 'caesar', "n't", 'served', 'lemon', 'wedge', 'low', 'opens', 'exactly', '11:30', 'shortly', 'arrive', 'bit', 'early', 'prepared', 'wait', 'outside', 'lounge', 'second', 'visit', 'keg', 'steakhouse', 'yonge/eglinton', 'unimpressed', 'last', 'time', 'time', "'s", 'slightly', 'better', 'least', 'steak', 'cooked', 'right', 'ca', "n't", 'pin', 'even', 'good', 'juicy', 'piece', 'new', 'york', 'cut', 'could', 'turn', 'dry', 'somewhat', 'bland', 'served', 'lukewarm', 'cook', 'right', 'medium', 'rare', 'redness', 'inside', 'still', 'though', 'ingredient', 'seasoning', 'steak'

I use Google's trained model to extract vectors.

In [26]:
X_gmodel = []

for review in X_nltk_new:
    vx = np.zeros(300)
    length = 0
    for w in review:
        if w in g_model.wv:
            length += 1
            vx += g_model[w]
    if length != 0: vx /= length
    X_gmodel.append(vx)
        

<ipython-input-26-d44cdbead26f>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if w in g_model.wv:


In [27]:
print(len(X_gmodel))
print(X_gmodel[0])

2991
[-0.00333445  0.04115454  0.01393951  0.12298004 -0.04709478 -0.00469536
  0.06399959 -0.08180204  0.05903501  0.1098259  -0.0238636  -0.10994214
  0.02001969 -0.00814393 -0.07882359  0.10122008  0.04601213  0.08382526
  0.01782823 -0.07004963  0.01598439  0.06013838  0.03976279 -0.01153185
  0.04035519 -0.02463523 -0.0639123   0.07411797  0.01485397  0.00044212
 -0.03641305  0.00071235  0.01297658  0.00047191  0.00121366 -0.03765829
  0.06853479 -0.0367369   0.01205842  0.05688488  0.04964095 -0.09290502
  0.12266873 -0.02565614 -0.007304   -0.07828741 -0.06979222  0.01756597
  0.00788619  0.02767111 -0.03055152  0.03888862 -0.01733739 -0.02401967
 -0.01580414  0.00856986  0.00474961 -0.03620944  0.02427391 -0.08058231
 -0.06478586  0.08781257 -0.10053659 -0.06595326  0.00296522 -0.03345597
 -0.07026498 -0.01084525 -0.01150617  0.05775703  0.08510416  0.00921234
  0.08737154  0.01916889 -0.15128901 -0.06450826 -0.00049878  0.06486876
  0.03309676  0.04146351  0.02671048 -0.063776

Train-test splitting.

In [28]:
X_train_gmodel, X_test_gmodel, y_train_gmodel, y_test_gmodel = train_test_split(X_gmodel, y, test_size=0.4, random_state=0)
print(len(X_train_gmodel), len(X_test_gmodel))
print(len(y_train_gmodel), len(y_test_gmodel))
print(y_train_gmodel)

1794 1197
1794 1197
[0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 2, 1, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 2, 2, 1, 1, 1, 1, 0, 2, 2, 1, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 0, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 2, 0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 2, 0, 2, 0, 1, 1, 0, 0, 1, 2, 2, 0, 2, 2, 1, 0, 2, 1, 2, 1, 0, 2, 0, 0, 2, 0, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 1, 1, 2, 0, 1, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 0, 2, 1, 0, 2, 2, 0, 0, 2, 0, 2, 2, 1, 0, 0, 1, 0, 2, 2, 2, 2, 2, 2, 0, 1, 0, 2, 0, 0, 1, 2, 2, 0, 1, 2, 0, 0, 1, 2, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 1, 0, 1, 1, 0, 2, 0, 2, 1, 1, 0, 2, 0, 2, 2, 0, 1, 2, 1, 1, 1, 2, 0, 2, 2, 2, 2, 1, 0, 1, 0, 0, 2, 1, 0, 0, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 2, 2, 2, 1, 2, 0, 0, 0, 1, 2, 0, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 0, 2, 0, 2, 2, 1, 2, 0, 1, 2, 0, 1, 0, 0, 1, 2, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2

I use all the methods all over again and print the necessary metric, just like in task 1.

In [29]:
ApplyModelAndPrintScores(lr_clf, X_train_gmodel, y_train_gmodel, X_test_gmodel, y_test_gmodel)

classifier score: 0.9565580618212197

accuracy: 0.9565580618212197

Confusion matrix
[[300  20   1]
 [  2 339  28]
 [  0   1 506]]

Precision Score per class
[0.99337748 0.94166667 0.94579439]

Average Precision Score
0.9572823134459284

Recall Score per class
[0.93457944 0.91869919 0.99802761]

Average Recall Score
0.9565580618212197

F1-score Score per class
[0.96308186 0.93004115 0.97120921]

Average F1 Score
0.9563387918889418


In [30]:
ApplyModelAndPrintScores(svm_clf, X_train_gmodel, y_train_gmodel, X_test_gmodel, y_test_gmodel)

classifier score: 0.9624060150375939

accuracy: 0.9624060150375939

Confusion matrix
[[304  17   0]
 [  2 341  26]
 [  0   0 507]]

Precision Score per class
[0.99346405 0.95251397 0.95121951]

Average Precision Score
0.9629472908091274

Recall Score per class
[0.9470405  0.92411924 1.        ]

Average Recall Score
0.9624060150375939

F1-score Score per class
[0.96969697 0.93810179 0.975     ]

Average F1 Score
0.9622032473748249


In [31]:
ApplyModelAndPrintScores(knn, X_train_gmodel, y_train_gmodel, X_test_gmodel, y_test_gmodel)

classifier score: 0.9682539682539683

accuracy: 0.9682539682539683

Confusion matrix
[[315   6   0]
 [  6 339  24]
 [  1   1 505]]

Precision Score per class
[0.97826087 0.97976879 0.95463138]

Average Precision Score
0.9687172354654899

Recall Score per class
[0.98130841 0.91869919 0.99605523]

Average Recall Score
0.9682539682539683

F1-score Score per class
[0.97978227 0.94825175 0.97490347]

Average F1 Score
0.9679958778159178


In [32]:
ApplyModelAndPrintScores(naive_bayes_model, X_train_gmodel, y_train_gmodel, X_test_gmodel, y_test_gmodel)

classifier score: 0.9440267335004177

accuracy: 0.9440267335004177

Confusion matrix
[[297  24   0]
 [  3 345  21]
 [  0  19 488]]

Precision Score per class
[0.99       0.88917526 0.95874263]

Average Precision Score
0.9456793524125872

Recall Score per class
[0.92523364 0.93495935 0.96252465]

Average Recall Score
0.9440267335004177

F1-score Score per class
[0.95652174 0.91149273 0.96062992]

Average F1 Score
0.944380674487942


In [33]:
ApplyModelAndPrintScores(dtree, X_train_gmodel, y_train_gmodel, X_test_gmodel, y_test_gmodel)

classifier score: 0.8972431077694235

accuracy: 0.8972431077694235

Confusion matrix
[[303  14   4]
 [ 25 319  25]
 [ 11  44 452]]

Precision Score per class
[0.89380531 0.84615385 0.93970894]

Average Precision Score
0.8985586517025735

Recall Score per class
[0.94392523 0.86449864 0.89151874]

Average Recall Score
0.8972431077694235

F1-score Score per class
[0.91818182 0.85522788 0.91497976]

Average F1 Score
0.8974187042191468
